# Site-specific LCIs

This notebook imports the raw LCI datasets for brine sites and made some modifications:

**Change 1: Formating LCI datasets metadata**

- Update some metadata of activities like "flow" to "reference product", "amount" to "production amount", add "database", etc.
- Some process names are used across projects (e.g., df_DLE_evaporation_ponds) - problematic to have everything in the same database - create unique activity names by adding the project name
- Generate unique code with wurst
- Location of foreground dataset is the name of the project (e.g., Chaerhan) - change to ISO location
- Reference product is missing in activities and exchanges

**Change 2: Update some LCI data and link to background processes and biosphere database**
- Link water flows to the new biosphere database with regionalized water flows
- Add "input" field for foreground processes
- Inputs of "heat production, natural gas, at industrial furnace >100kW" and "machine operation, diesel, >= 74.57 kW, high load factor" have same location as project due to regionalization - change location to RoW

In [1]:
import brightway2 as bw
import bw2io
import wurst
import pandas as pd
from pathlib import Path
import shutil
import copy
from utils import relink_to_regionalized_water

In [2]:
BW_PROJECT = 'lithium_brine'
bw.projects.set_current(BW_PROJECT)

DB_NAME = "lithium_brine_projects"
EI_NAME = "ecoinvent-3.10-cutoff"
BIO_NAME = "ecoinvent-3.10-biosphere"
BIO_WATER_NAME = "biosphere water regionalized"

technosphere = lambda x: x["type"] == "technosphere"
biosphere = lambda x: x["type"] == "biosphere"
production = lambda x: x["type"] == "production"
economic_flows = lambda x: x["type"] in ["technosphere", "production"]

## Import ecoinvent database and lithium LCIs

In [3]:
# Import ecoinvent database with wurst (i.e., list of dictionary, each dict being a dataset)
try:
  len(ei_db)
except NameError:
  ei_db = wurst.extract_brightway2_databases(EI_NAME)

# Import biosphere database
bio_db = [ds for ds in bw.Database(BIO_NAME)]

Getting activity data


100%|██████████| 23523/23523 [00:00<00:00, 81714.73it/s] 


Adding exchange data to activities


100%|██████████| 743409/743409 [01:02<00:00, 11893.78it/s]


Filling out exchange data


100%|██████████| 23523/23523 [00:02<00:00, 8354.66it/s] 


In [4]:
# Import lithium LCIs
lci_raw_path = Path("../inventories/raw")
lci_files = [file for file in lci_raw_path.glob("*.xlsx")] + [file for file in lci_raw_path.glob("*.xls")]

def import_lci_dataset(lci_path, ei_name, bio_name):
    lci = bw.ExcelImporter(lci_path)
    lci.apply_strategies(verbose=False)
    lci.match_database(ei_name, fields=('name', 'reference product', 'unit', 'location'))
    lci.match_database(bio_name, fields=('name', 'unit', 'categories'))
    return lci

In [5]:
# key: name of the project \
# value: dictionary containing the LCI
lci_raw_dict = {}
for file in lci_files:
    lci_project = import_lci_dataset(file, EI_NAME, BIO_NAME)
    lci_raw_dict.update({lci_project.db_name: lci_project.data})

Extracted 1 worksheets in 0.10 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Extracted 1 worksheets in 0.05 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Extracted 1 worksheets in 0.04 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Extracted 1 worksheets in 0.04 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Extracted 1 worksheets in 0.04 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Extracted 1 worksheets in 0.02 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Extracted 1 worksheets in 0.04 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Extracted 1 worksheets in 0.03 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


## Change 1: Formating LCI datasets metadata

In [6]:
list(lci_raw_dict.keys())

['Chaerhan',
 'East Taijinar',
 'Fenix',
 'Kachi',
 'Lakkor Tso',
 'Maricunga',
 'Pozuelos',
 'Qinghai Yiliping',
 'Sal de los Angeles',
 'Sal de Vida',
 'Salar de Antofalla',
 'Salar de Arizaro',
 'Salar de Atacama',
 'Salar de Cauchari-Olaroz',
 'Salar de Centenario',
 'Salar de Olaroz',
 'Salar de Pastos Grandes',
 'Salar de Rio Grande',
 'Salar de Salinas Grandes',
 'Salar de Tolillar',
 'Salar de Uyuni',
 'Salar del Hombre Muerto North',
 'Salar del Rincon',
 'Salton Sea',
 'Silver Peak',
 'Tres Quebradas',
 'Upper Rhine Graben',
 'Zhabuye']

In [7]:
projects_locations = {
    'Chaerhan': "CN",
    'Fenix': "AR",
    'Kachi': "AR",
    'Maricunga': "CL",
    'Pozuelos': "AR",
    'Qinghai Yiliping': "CN",
    'Sal de los Angeles': "AR",
    'Sal de Vida': "AR",
    'Salar de Antofalla': "AR",
    'Salar de Arizaro': "AR",
    'Salar de Atacama': "CL",
    'Salar de Cauchari-Olaroz': "AR",
    'Salar de Centenario': "AR",
    'Salar de Olaroz': "AR",
    'Salar de Pastos Grandes': "AR",
    'Salar de Rio Grande': "AR",
    'Salar de Salinas Grandes': "AR",
    'Salar de Tolillar': "AR",
    'Salar de Uyuni': "BO",
    'Salar del Hombre Muerto North': "AR",
    'Salar del Rincon': "AR",
    'Salton Sea': "US",
    'Silver Peak': "US",
    'Tres Quebradas': "AR",
    'Upper Rhine Graben': "DE",
    'Zhabuye': "CN",
    "East Taijinar": "CN",
    "Lakkor Tso": "CN"
}

In [8]:
unique_names = {}

for project in lci_raw_dict:
    
    # Create unique names for datasets associated with the project
    unique_names = {ds["name"]: ds["name"] + "_" + project for ds in lci_raw_dict[project]}

    # Some changes in datasets metadata
    for ds in lci_raw_dict[project]:
        if "flow" in ds:
            ds['reference product'] = ds.pop('flow')
        if "amount" in ds:
            ds['production amount'] = ds.pop('amount')
        ds["database"] = DB_NAME

        # Change to unique activity name
        if ds["name"] in unique_names.keys():
            ds.update({"name": unique_names[ds["name"]]})

        # Generate unique code
        ds.update({"code": wurst.filesystem.get_uuid()})

        # Change location of activity
        if ds["location"] in projects_locations.keys():
            ds.update({"location": projects_locations[ds["location"]]})

    # Make some changes in the exchange flows:
    for ds in lci_raw_dict[project]:
        for exc in filter(economic_flows, ds["exchanges"]):

            # Add product and input to production exchanges:
            # "product" is used instead of "reference product" for wurst
            if exc in filter(production, ds["exchanges"]):
                exc.update({
                        "product": ds["reference product"],
                        "input": (ds["database"], ds["code"])
                    })

            # Change exchange name to unique name
            if exc["name"] in unique_names.keys():
                exc.update({"name": unique_names[exc["name"]]})
                
            if exc["location"] in projects_locations.keys():
                exc.update({"location": projects_locations[exc["location"]]})

## Change 2: Link to background processes and biosphere database

In [9]:
# Find if there is any background dataset that is not available in ecoinvent
list_of_foreground_ds = [ds["name"] for project in lci_raw_dict for ds in lci_raw_dict[project]]
list_of_ei_ds = [(ds["name"], ds["location"]) for ds in bw.Database(EI_NAME)]

no_ecoinvent_lcis = []

for project in lci_raw_dict:
    for ds in lci_raw_dict[project]:
        for exc in filter(technosphere, ds["exchanges"]):
            # Do this only for background datasets; i.e., datasets that are not in the project associated datasets
            if exc["name"] not in list_of_foreground_ds:
                exc_ds = [ds for ds in list_of_ei_ds if ds[0] == exc["name"] and ds[1] == exc["location"]]
                if len(exc_ds) == 0:
                    no_ecoinvent_lcis.append((exc["name"], exc["location"]))

no_ecoinvent_lcis = list(set(no_ecoinvent_lcis))

In [10]:
no_ecoinvent_lcis

[('market for wastewater, average', 'US'),
 ('market for wastewater, average', 'CN'),
 ('market for wastewater, average', 'DE'),
 ('market for wastewater, average', 'BO'),
 ('market for wastewater, average', 'AR'),
 ('heat production, natural gas, at industrial furnace >100kW', 'CN'),
 ('sodium hydroxide to generic market for neutralising agent', 'GLO'),
 ('heat production, natural gas, at industrial furnace >100kW', 'US'),
 ('heat production, natural gas, at industrial furnace >100kW', 'BO'),
 ('heat production, natural gas, at industrial furnace >100kW', 'DE'),
 ('machine operation, diesel, >= 74.57 kW, high load factor', 'CL'),
 ('heat production, natural gas, at industrial furnace >100kW', 'AR'),
 ('market for soda ash, light', 'GLO'),
 ('deep well drilling, for deep geothermal power reg', 'DE'),
 ('machine operation, diesel, >= 74.57 kW, high load factor', 'CN'),
 ('machine operation, diesel, >= 74.57 kW, high load factor', 'US'),
 ('market for wastewater, average', 'CL'),
 ('mach

In [11]:
for project in lci_raw_dict:
    
    for ds in lci_raw_dict[project]:
        # Add product and input to technosphere exchanges:
        for exc in filter(technosphere, ds["exchanges"]):

            # Find datasets for foreground exchanges
            if exc["name"] in list_of_foreground_ds:
                exc_ds = [d for project in lci_raw_dict for d in lci_raw_dict[project] if d["name"] == exc["name"]]
                
             # Find dataset for background exchanges:
            else:
                
                # Change several locations to RoW:
                if exc["name"] in ["heat production, natural gas, at industrial furnace >100kW",
                                   "market for wastewater, average",
                                   "market for soda ash, light",
                                   "sodium hydroxide to generic market for neutralising agent"]:
                    exc.update({"location": "RoW"})  

                # Change location of mechine diesel to GLO:
                if exc["name"] == "machine operation, diesel, >= 74.57 kW, high load factor":
                    exc.update({"location": "GLO"})          
      
                # Change name and location of 'deep well drilling, for deep geothermal power reg'
                if exc["name"] == "deep well drilling, for deep geothermal power reg":
                    exc.update({"name": "deep well drilling, for deep geothermal power"})
                    if exc["location"] == "US-WECC":
                        exc.update({"location": "US-HICC"})

                exc_ds = [ds for ds in bw.Database(EI_NAME) if ds["name"] == exc["name"] and 
                                                               ds["location"] == exc["location"] and
                                                               ds["unit"] == exc["unit"]]
                
            if len(exc_ds) > 1:
                raise ValueError("More than one dataset for exchange", (exc["name"], exc["location"])) 
                
            if len(exc_ds) == 0:
                raise ValueError("LCI dataset not found for", (exc["name"], exc["location"]))
            
            # "product" is used instead of "reference product" for wurst
            exc.update({
                    "product": exc_ds[0]["reference product"],
                    "input": (exc_ds[0]["database"], exc_ds[0]["code"])
                })

In [13]:
# Find biosphere flows that are not in the biosphere database
list_of_bio_ds = [(ds["name"], ds["categories"]) for ds in bw.Database(BIO_NAME)]
no_biosphere_flows = []

for project in lci_raw_dict:
    for ds in lci_raw_dict[project]:
        for exc in filter(biosphere, ds["exchanges"]):
            if (exc["name"], exc["categories"])  not in list_of_bio_ds:
                no_biosphere_flows.append(exc["name"])

In [14]:
no_biosphere_flows

['Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium',
 'Sodium']

In [15]:
# Update "Sodium" to "Sodium I"
sodium_i_ds = [ds for ds in bw.Database(BIO_NAME) if ds["name"]=="Sodium I" and ds["categories"]==("water",)][0]

for project in lci_raw_dict:
    for ds in lci_raw_dict[project]:
        for exc in filter(biosphere, ds["exchanges"]):
            if exc["name"] == "Sodium":
                exc["name"] = "Sodium I"
                exc["input"] = sodium_i_ds.key

### Link water flows to regionalized biosphere

In [16]:
water_flows_regionalized = [act for act in bw.Database(BIO_WATER_NAME)]

for project in lci_raw_dict:
    for ds in lci_raw_dict[project]:
        relink_to_regionalized_water(water_flows_regionalized, ds, location=project)

### Export LCIs

In [17]:
lci_all = []
for project in lci_raw_dict:
    lci_all.extend(lci_raw_dict[project])

In [18]:
if DB_NAME in bw.databases:
    del bw.databases[DB_NAME]
wurst.write_brightway2_database(lci_all, DB_NAME)

Vacuuming database 
505 datasets
2611 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 04/05/2025 22:53:32
  Finished: 04/05/2025 22:53:33
  Total time elapsed: 00:00:00
  CPU %: 91.90
  Memory %: 20.79
Created database: lithium_brine_projects


In [19]:
# Export inventories to excel file:
export_path = bw2io.export.excel.write_lci_excel(DB_NAME)
filepath = Path("../inventories")
shutil.copy(export_path, filepath)

'..\\inventories\\lci-lithium_brine_projects.xlsx'